<a href="https://colab.research.google.com/github/Pragatheeswara5/Machine-Learning/blob/master/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [ ]:
CSV_COLUMN_NAMES=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species'] #info about each flower
SPECIES=['Setosa','Versicolor','Virginica'] #types of species

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

573/573 [==============================] - 0s 0us/step


In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train_y=train.pop("Species")
test_y=test.pop("Species")
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
train.shape



(120, 4)

In [ ]:
def input_fn(features,labels,training=True,batch_size=256):
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))

  if training:
    dataset=dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)


In [ ]:
my_feature_columns=[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
#creating a model--- deep neuarl network model
classifier=tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,

    #two hidden layers of 30 and 10 nodes respectively
    hidden_units=[30,10],

    #the model must choose between 3 classes
    n_classes=3
)

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000
)

Instructions for updating:
Use standard file utilities to get mtimes.


In [ ]:
eval_result=classifier.evaluate(input_fn=lambda: input_fn(train, train_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.933



In [ ]:
def input_fn(features,batch_size=256):
  # Convert the inputs to a dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}

print("Please type numeric values as prompted.")

for feature in features:
  valid=True
  while valid:
    val=input(feature + ": ")
    if not val.isdigit(): valid=False

  predict[feature]=[float(val)]

predictions=classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id=pred_dict['class_ids'][0]
  probability=pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(
      SPECIES[class_id], 100*probability
  ))


Please type numeric values as prompted.
SepalLength: 2.1
SepalWidth: 1.4
PetalLength: 3.4
PetalWidth: 4.5


Exception ignored in: <generator object Estimator.predict at 0x7f6cc9e38ac0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow_estimator/python/estimator/estimator.py", line 618, in predict
  File "/usr/lib/python3.10/contextlib.py", line 153, in __exit__
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/ops.py", line 5931, in get_controller
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/context.py", line 272, in pop
IndexError: pop from empty list


{'logits': array([-0.9756547 , -1.4172765 ,  0.68797857], dtype=float32), 'probabilities': array([0.14447846, 0.0928986 , 0.7626229 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (76.3%)
